In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as sc

from DeepFMKit.experiments import Experiment
from DeepFMKit.factories import StandardWDFMIExperimentFactory
from DeepFMKit.waveforms import second_harmonic_distortion

exp = Experiment(description="W-DFMI Bias Correction Verification")
exp.add_axis('distortion_amp', np.linspace(0.00, 0.1, 11))
exp.set_static({
    'm_main': 20.3,
    'm_witness': 0.5,
})
exp.n_trials = 50
exp.add_stochastic_variable(
    'waveform_kwargs', 
    lambda dist_amp: {'distortion_amp': dist_amp, 'distortion_phase': np.random.uniform(0, 2*np.pi)},
    depends_on='distortion_amp'
)

exp.set_config_factory(
    StandardWDFMIExperimentFactory(waveform_function=second_harmonic_distortion)
)

exp.add_analysis(name='wdfmi_fit', fitter_method='wdfmi_ortho', result_cols=['tau'])
exp.add_analysis(name='nls_fit', fitter_method='nls', result_cols=['m'], fitter_kwargs={'ndata': 50})

print("Experiment configured successfully using the final, robust factory pattern.")

if __name__ == "__main__":
    results = exp.run()

In [ ]:
df_cal_error_frac = 0.01

# --- Ground Truth and Experimenter's Estimate ---
sample_params = exp.get_params_for_point(axis_idx=0)
configs = exp.config_factory(sample_params)
opd_true = configs['main_ifo_config'].meas_arml - configs['main_ifo_config'].ref_arml
tau_true = opd_true / sc.c
wavelength = configs['laser_config'].wavelength
df_true = configs['laser_config'].df

df_est = df_true * (1.0 - df_cal_error_frac)
print(f"True df: {df_true/1e9:.4f} GHz")
print(f"Simulated experimenter's df (with {df_cal_error_frac*100:.1f}% error): {df_est/1e9:.4f} GHz")

# --- Extract and Process Fitter Results ---
wdfmi_tau_all = results['wdfmi_fit']['tau']['all_trials']
nls_m_all = results['nls_fit']['m']['all_trials']
nls_tau_all = nls_m_all / (2 * np.pi * df_est)

wdfmi_len_err_all = (wdfmi_tau_all - tau_true) * sc.c
nls_len_err_all = (nls_tau_all - tau_true) * sc.c

wdfmi_len_err_mean = np.nanmean(wdfmi_len_err_all, axis=-1)
wdfmi_len_err_std = np.nanstd(wdfmi_len_err_all, axis=-1)
nls_len_err_mean = np.nanmean(nls_len_err_all, axis=-1)
nls_len_err_std = np.nanstd(nls_len_err_all, axis=-1)

# --- Plotting ---
fig, ax = plt.subplots(figsize=(12, 7))
dist_axis_pct = results['axes']['distortion_amp'] * 100

nls_label = f'Standard DFMI (with {df_cal_error_frac*100:.0f}% Cal. Error)'
ax.errorbar(dist_axis_pct, nls_len_err_mean * 1e6, yerr=nls_len_err_std * 1e6,
            fmt='o-', capsize=5, color='tab:red', label=nls_label)

ax.errorbar(dist_axis_pct, wdfmi_len_err_mean * 1e6, yerr=wdfmi_len_err_std * 1e6,
            fmt='s-', capsize=5, color='tab:green', label='W-DFMI (Immune to Cal. Error)')

ambiguity_limit_um = (wavelength / 2) * 1e6
ax.axhline(ambiguity_limit_um, color='k', linestyle='--', linewidth=2, label=r'Ambiguity Limit ($\pm\lambda_0/2$)')
ax.axhline(-ambiguity_limit_um, color='k', linestyle='--', linewidth=2)
ax.axhline(0, color='k', linestyle='-', linewidth=1, alpha=0.5)

ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
ax.set_ylabel(r"Absolute Length Error, $\delta(\Delta l)$ ($\mu$m)", fontsize=14)
ax.set_title(exp.description, fontsize=16)
ax.grid(True, which='both', linestyle=':')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()